In [ ]:
from scipy.sparse import bsr_array
import numpy as np
X = np.loadtxt('datasets/ipsc.txt')

In [ ]:
X_sparse = bsr_array(X)

The thing is: going to a sparse matrix doesn't actually help us *at all* with the data, because we don't expect the data to have tons of zeros. But it's worth a try, I suppose.

How do we multiply sparse matrices to a given power? We can do `@` for single powers, which works fairly well, and we can loop this following the log to achieve a matrix power to a given `t`.
But because the number of non-zero entries multiplies with matrix multiplication, we have to do the multiplications in the *landmarked* realm.

In [ ]:
import graphtools
G = graphtools.Graph(X, n_landmark=500)

Interestingly, graphtools makes full use of all of the CPU cores. That's good programming.

In [ ]:
sP = G.P # our sparse matrix
P_landmarks = G.landmark_op

In [ ]:
# saving these for future usage
import pickle
with open('datasets/ipsc_sP.pickle','wb') as f:
  pickle.dump(sP,f)
with open('datasets/ipsc_P_landmarks.pickle','wb') as f:
  pickle.dump(P_landmarks,f)

In [ ]:
G.transitions # is this a sparse matrix?
trans = bsr_array(G.transitions)
full_transitions = G.transitions
P_l_bsr = bsr_array(P_landmarks)

In [ ]:
P_l_bsr

<500x500 sparse matrix of type '<class 'numpy.float64'>'
	with 81612 stored elements (blocksize = 1x1) in Block Sparse Row format>

In [ ]:
with open('datasets/ipsc_transitions.pickle','wb') as f:
  pickle.dump(trans,f)

In [ ]:
trans

<220450x500 sparse matrix of type '<class 'numpy.float64'>'
	with 110225000 stored elements (blocksize = 2x2) in Block Sparse Row format>

In [ ]:
tolerance = 0.0001
full_transitions[full_transitions < tolerance] = 0

/Users/adjourner/miniforge3/envs/graphcurvature/lib/python3.10/site-packages/scipy/sparse/_compressed.py:291: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using < is inefficient, try using >= instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)
/Users/adjourner/miniforge3/envs/graphcurvature/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
full_transitions

<220450x500 sparse matrix of type '<class 'numpy.float64'>'
	with 110225000 stored elements in Compressed Sparse Row format>

In [ ]:
trans = bsr_array(full_transitions)
trans

<220450x500 sparse matrix of type '<class 'numpy.float64'>'
	with 110225000 stored elements (blocksize = 2x2) in Block Sparse Row format>

Just to see how fast this matrix multiplication is:

(Unlike G.P, which has a lot of zeros, the transition matrix has values almost everywhere.)

In [ ]:
trans @ P_landmarks

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.61419128e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.61432691e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.61432691e-05],
       ...,
       [0.00000000e+00, 1.58368284e-06, 1.01428970e-05, ...,
        1.81705757e-04, 1.38433002e-05, 1.99467440e-08],
       [0.00000000e+00, 9.30869523e-05, 0.00000000e+00, ...,
        1.26967055e-08, 8.85121684e-06, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.67227408e-06, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
#| export core
def matpower(A,t):
  At = A
  two_powers = int(np.floor(np.log2(t)))
  remainder = t - 2**two_powers
  for i in range(two_powers):
    At = At @ At
  for j in range(remainder):
    At = At @ A
  return At

In [ ]:
matpower(P_l_bsr,3)

<500x500 sparse matrix of type '<class 'numpy.float64'>'
	with 250000 stored elements (blocksize = 1x1) in Block Sparse Row format>

In [ ]:
A2_landmark_part_1 = trans @ matpower(P_landmarks,2)
A2_landmark_part_1
A2_landmark = A2_landmark_part_1 @ trans.T

In [ ]:
A2_real = sP @ sP

In [ ]:
sP

NameError: name 'sP' is not defined

In [ ]:
A8_manual = (sP @ sP)

In [ ]:
A8_manual 

NameError: name 'A8_manual' is not defined

In [ ]:
A8_manual = A8_manual @ A8_manual

: 

: 

For excessive numbers of points, diffusing between each of them repeatedly ceases to make sense -- hence the appeal of *landmarking*. PHATE uses landmarks to approximate large powers of the diffusion matrix, by (in effect) subsampling to a smaller number of "landmarks" and building a diffusion matrix between these.

In PHATE, the landmark matrix is powered and used by itself. This could produce a laziness measure of the landmarked points (which works quickly for up to 10,000 points). But how to extend this laziness back to the other points?

One obvious solution is to diffuse the laziness of the landmark points down to each of the other points.